1) We're going to simulate a glider using Newton-Euler dynamics

In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating environment at `~/devel/dynamics-simulation-16-715/HW2_solutions/Project.toml`


In [2]:
using LinearAlgebra
using StaticArrays
using ForwardDiff
using Plots
plotlyjs()

WebIO._IJuliaInit()

Plots.PlotlyJSBackend()

In [3]:
#Some standard functions for dealing with rotation matrices and quaternions from the class notes

function hat(ω)
    return [0 -ω[3] ω[2];
            ω[3] 0 -ω[1];
            -ω[2] ω[1] 0]
end

function L(q)
    [q[1] -q[2:4]'; q[2:4] q[1]*I + hat(q[2:4])]
end

function R(q)
    [q[1] -q[2:4]'; q[2:4] q[1]*I - hat(q[2:4])]
end

H = [zeros(1,3); I];

In [4]:
#Glider parameters

g = 9.81; #Gravitational acceleration (m/s^2)
ρ = 1.225; #Air density at 20C (kg/m^3)
m = .484; #Mass of plane (kg)

# Inertia (kg*m^2)
Jx  = 0.003922;
Jy  = 0.015940;
Jz  = 0.019340;
Jxz = 0.000441;
J = [Jx 0 Jxz;
     0  Jy  0;
     Jxz 0 Jz];

# Main Wing
b = 86.4/100; #wing span (m)
cr = 26/100; #root chord (m)
ct = 15.2/100; #tip chord (m)
cm = (ct + cr)/2; #mean wing chord (m)
S = b*cm; #planform area of wing (m^2)
Ra = b^2/S; #wing aspect ratio (dimensionless)
Rt = ct/cr; #wing taper ratio (dimensionless)
r_wing1 = [0; (b/6)*(1+2*Rt)/(1+Rt); 0]; #vector from CoM to wing center-of-pressure (m)
r_wing2 = [0; -(b/6)*(1+2*Rt)/(1+Rt); 0];
ϵ_ail = 0.45 #flap effectiveness (dimensionless)

# Elevator
b_ele = 18.2/100; #elevator span (m)
cr_ele = 15.2/100; #elevator root chord (m)
ct_ele = 14/100; #elevator tip chord (m)
cm_ele = (ct_ele + cr_ele)/2; #mean elevator chord (m)
S_ele = b_ele*cm_ele; #planform area of elevator (m^2)
Ra_ele = b_ele^2/S_ele; #elevator aspect ratio (dimensionless)
r_ele = [-45/100; 0; 0]; #vector from CoM to elevator center-of-pressure (m)
ϵ_ele = 0.8 #flap effectiveness (dimensionless)

# Rudder
b_rud = 21.6/100; #rudder span (m)
cr_rud = 20.4/100; #rudder root chord (m)
ct_rud = 12.9/100; #rudder tip chord (m)
cm_rud = (ct_rud + cr_rud)/2; #mean rudder chord (m)
S_rud = b_rud*cm_rud; #planform area of rudder (m^2)
Ra_rud = b_rud^2/S_rud; #rudder aspect ratio (dimensionless)
r_rud = [-48/100; 0; -3/100] #vector from CoM to rudder center-of-pressure (m)
ϵ_rud = 0.7 #flap effectiveness (dimensionless)

# Lift and drag polynomial coefficients from wind tunnel data
Clcoef = [38.779513049043175; 0.0; 19.266141214863080; 0.0; -13.127972418509980; 0.0; 3.634063117174400; 0.0];
Cdcoef = [3.607550808703421; 0.0; -4.489225907857385; 0.0; 3.480420330498847; 0.0; 0.063691497636087];

In [5]:
#These functions calculate lift and drag coefficients as a function of angle-of-attack
#(or side-slip angle in the case of the rudder)

function Cl_wing(α)
    Cl = Clcoef'*(α.^((length(Clcoef)-1):-1:0))
end

function Cd_wing(α)
    Cd = Cdcoef'*(α.^((length(Cdcoef)-1):-1:0))
end

function Cl_ele(α)
    Cl = (pi/2)*Ra_ele*α
end

function Cd_ele(α)
    Cd = (pi/4)*Ra_ele*α*α
end

function Cl_rud(β)
    Cl = (pi/2)*Ra_rud*β
end

function Cd_rud(β)
    Cd = (pi/4)*Ra_rud*β*β
end

function aoa(v)
    #Calculate angle-of-attack from body-frame velocity
    
    ### Solution
    return atan(v[3],v[1])
end

function ss(v)
    #Calculate side-slip angle from body-frame velocity
    #This is similar to angle-of-attack, but for yaw rather than pitch
    return atan(v[2],v[1])
end

ss (generic function with 1 method)

In [6]:
#Plot Cl and Cd for the main wing

α = LinRange(-20*(pi/180),20*(pi/180),100)
cls = zeros(length(α))
cds = zeros(length(α))
for k = 1:length(α)
    cls[k] = Cl_wing(α[k])
    cds[k] = Cd_wing(α[k])
end
plot((180/pi).*α,cls)
plot!((180/pi).*α,cds)

In [7]:
#Now let's compute the aerodynamic forces and torques on the glider from each lifting surface

function aero_forces(v,ω,u)
    
    # Template code
    #v_wing1 = #Calculate the velocity of the right wing's center of pressure, accounting for the glider's angular velocity
    #α_wing1 = aoa(v_wing1)
    #α_eff_wing1 = #Calculate effective angle of attack accounting for aileron defflection
    #L_wing1 = #Calculate lift component in the wind frame
    #D_wing1 = #Calculate drag component in the wind frame
    #F_wing1 = αrotate(α_wing1,L_wing1,D_wing1) #rotate lift and drag into the glider's body frame
    #τ_wing1 = #Calculate torque about the glider's CoM due to lift and drag
    
    #v_wing2 = #Calculate the velocity of the left wing's center of pressure, accounting for the glider's angular velocity
    #α_wing2 = aoa(v_wing2)
    #α_eff_wing2 = #Calculate effective angle of attack accounting for aileron defflection
    #L_wing2 = #Calculate lift component in the wind frame
    #D_wing2 = #Calculate drag component in the wind frame
    #F_wing2 = αrotate(α_wing2,L_wing2,D_wing2) #rotate lift and drag into the glider's body frame
    #τ_wing2 = #Calculate torque about the glider's CoM due to lift and drag
    
    #v_ele = #Calculate the velocity of the elevator's center of pressure, accounting for the glider's angular velocity
    #α_ele = aoa(v_ele)
    #α_eff_ele = #Calculate effective angle of attack accounting for elevator defflection
    #L_ele = #Calculate lift component in the wind frame
    #D_ele = #Calculate drag component in the wind frame
    #F_ele = αrotate(α_ele,L_ele,D_ele) #rotate lift and drag into the glider's body frame
    #τ_ele = #Calculate torque about the glider's CoM due to lift and drag
    
    #v_rud = #Calculate the velocity of the elevator's center of pressure, accounting for the glider's angular velocity
    #β_rud = ss(v_rud) #side slip angle of rudder
    #β_eff_rud = #Calculate effective side slip accounting for rudder defflection
    #L_rud = #Calculate lift component in the wind frame
    #D_rud = #Calculate drag component in the wind frame
    #F_rud = βrotate(β_rud,L_rud,D_rud) #rotate lift and drag into the glider's body frame
    #τ_rud = #Calculate torque about the glider's CoM due to lift and drag
    
    ### Solution
    v_wing1 = v + hat(ω)*r_wing1
    α_wing1 = aoa(v_wing1)
    α_eff_wing1 = α_wing1 - ϵ_ail*u[1]
    L_wing1 = 0.5*ρ*S*Cl_wing(α_eff_wing1)*(v_wing1'*v_wing1)
    D_wing1 = 0.5*ρ*S*Cd_wing(α_eff_wing1)*(v_wing1'*v_wing1)
    F_wing1 = αrotate(α_wing1,L_wing1,D_wing1)
    τ_wing1 = hat(r_wing1)*F_wing1
    
    v_wing2 = v + hat(ω)*r_wing2
    α_wing2 = aoa(v_wing2)
    α_eff_wing2 = α_wing2 + ϵ_ail*u[1]
    L_wing2 = 0.5*ρ*S*Cl_wing(α_eff_wing2)*(v_wing2'*v_wing2)
    D_wing2 = 0.5*ρ*S*Cd_wing(α_eff_wing2)*(v_wing2'*v_wing2)
    F_wing2 = αrotate(α_wing2,L_wing2,D_wing2)
    τ_wing2 = hat(r_wing2)*F_wing2
    
    v_ele = v + hat(ω)*r_ele
    α_ele = aoa(v_ele)
    α_eff_ele = α_ele - ϵ_ele*u[2]
    L_ele = 0.5*ρ*S*Cl_ele(α_eff_ele)*(v_ele'*v_ele)
    D_ele = 0.5*ρ*S*Cd_ele(α_eff_ele)*(v_ele'*v_ele)
    F_ele = αrotate(α_ele,L_ele,D_ele)
    τ_ele = hat(r_ele)*F_ele
    
    v_rud = v + hat(ω)*r_rud
    β_rud = ss(v_rud)
    β_eff_rud = β_rud + ϵ_rud*u[3]
    L_rud = 0.5*ρ*S*Cl_rud(β_eff_rud)*(v_rud'*v_rud)
    D_rud = 0.5*ρ*S*Cd_rud(β_eff_rud)*(v_rud'*v_rud)
    F_rud = βrotate(β_rud,L_rud,D_rud)
    τ_rud = hat(r_rud)*F_rud
    
    F = F_wing1 + F_wing2 + F_ele + F_rud
    τ = τ_wing1 + τ_wing2 + τ_ele + τ_rud
    
    return F, τ
end

function αrotate(α,L,D)
    #Rotate by angle of attack from wind frame into body frame
    
    ### Solution
    rrot = [cos(α) 0  -sin(α);
              0    1    0;
            sin(α) 0  cos(α)]*[-D; 0; -L];
end

function βrotate(β,L,D)
    #Rotate by sideslip angle from wind frame into body frame
    
    ### Solution
    rrot = [cos(β) -sin(β) 0;
            sin(β)  cos(β) 0;
              0       0    1]*[-D; -L; 0];
end

βrotate (generic function with 1 method)

In [8]:
#Using the aerodynamic forces and torques, implement the Newton-Euler dynamics for the glider

function glider_dynamics_q(x,u)
    #Unpack state vector
    r = x[1:3] #N frame
    q = x[4:7] #B to N
    v = x[8:10] #B frame
    ω = x[11:13] #B frame
    
    ### Solution
    Q = H'*L(q)*R(q)'*H
    
    #Forces
    F_aero, τ_aero = aero_forces(v,ω,u)
    F = F_aero + Q'*[0; 0; -m*g]
    τ = τ_aero
    
    #Kinematics
    ṙ = Q*v
    q̇ = 0.5*L(q)*H*ω
    
    #Dynamics
    v̇ = F/m - hat(ω)*v
    ω̇ = J\(τ - hat(ω)*J*ω)
    
    ẋ = [ṙ; q̇; v̇; ω̇]
end

glider_dynamics_q (generic function with 1 method)

In [9]:
#Nominal Initial Conditions (leave this alone)

#note that the conventional body-frame coordinate system for airplanes is:
# x pointing out the nose
# y pointing out the right wing
# z pointing out the floor

r0 = [-10.0; 0; 10.0]
q0 = [cos(pi/2); sin(pi/2); 0; 0] #this rotates the airplane so it is right-side up
v0 = [10.0; 0; 0]
ω0 = zeros(3)
x0 = [r0; q0; v0; ω0];
u0 = zeros(3);

In [10]:
#We're now going to find trim conditions for forward flight. This is similar to an equilibrium.
#We define trim as motion at constant velocity and constant attitude (v̇=0 and ω=0)
#In general, this will require non-zero pitch, vertical velocity, and elevator inputs

function trim_error(z)
    #z = [pitch; vz; elevator]
    
    ### Solution
    q = L(q0)*[sqrt(1-z[1]^2); 0; z[1]; 0]
    v = v0 + [0; 0; z[2]]
    x = [r0; q; v; ω0]
    u = [0; z[3]; 0]
    ẋ = glider_dynamics_q(x,u)
    
    error = ẋ[[8;10;12]] #from symmetry considerations, we only need to worry about v̇x, v̇z, and ω̇y
end

trim_error (generic function with 1 method)

In [11]:
#Use Newton's method to solve for trim conditions

### Solution
z = [0.0; 0.0; 0.0] #initial guess
e = trim_error(z)
while maximum(abs.(e)) > 1e-12
    de = ForwardDiff.jacobian(trim_error,z)
    z .= z-de\e
    e = trim_error(z)
end

q0 = L(q0)*[sqrt(1-z[1]^2); 0; z[1]; 0]
v0 = v0 + [0; 0; z[2]]
x0 = [r0; q0; v0; ω0]
u0 = [0; z[3]; 0];

In [12]:
#Simulate the glider from the given initial conditions for 5 seconds at 50Hz

Tf = 5.0
h = 1.0/50
tsamp = 0:h:Tf
N = length(tsamp)

xhist_mid = zeros(13,N)
xhist_mid[:,1] .= x0

xhist_rk4 = zeros(13,N)
xhist_rk4[:,1] .= x0

#Implement explicit midpoint with the standard hack of normalizing the quaternion at each step
function normalized_mid_step(xk,uk)
    
    ### Solution
    ẋ1 = glider_dynamics_q(xk,uk)
    ẋ2 = glider_dynamics_q(xk.+0.5*h*ẋ1,uk)
    xn = xk + h*ẋ2
    xn[4:7] .= xn[4:7]/norm(xn[4:7])
    
    return xn
end

#Implement RK4 with the standard hack of normalizing the quaternion at each step
function normalized_rk4_step(xk,uk)
    
    ### Solution
    ẋ1 = glider_dynamics_q(xk,uk)
    ẋ2 = glider_dynamics_q(xk.+0.5*h*ẋ1,uk)
    ẋ3 = glider_dynamics_q(xk.+0.5*h*ẋ2,uk)
    ẋ4 = glider_dynamics_q(xk.+h*ẋ3,uk)
    #xn = xk + h*ẋ2
    xn = xk + (h/6.0)*(ẋ1 + 2*ẋ2 + 2*ẋ3 + ẋ4)
    xn[4:7] .= xn[4:7]/norm(xn[4:7])
    
    return xn
end

for k = 1:(N-1)
    xhist_mid[:,k+1] .= normalized_rk4_step(xhist_mid[:,k],u0)
    xhist_rk4[:,k+1] .= normalized_rk4_step(xhist_rk4[:,k],u0)
end

In [13]:
#Setup code for the MeshCat visualizer
using RobotDynamics, Rotations
    
struct Glider{R} <: RigidBody{R}
    n::Int
    m::Int
end
RobotDynamics.control_dim(::Glider) = 3

model = Glider{UnitQuaternion}(13,3)

using TrajOptPlots
using MeshCat
using FileIO, MeshIO, Colors, GeometryBasics, CoordinateTransformations
function TrajOptPlots._set_mesh!(vis, model::Glider)
    obj = joinpath(@__DIR__, "piper_scaled.obj")
    robot_obj = FileIO.load(obj)
    mat = MeshPhongMaterial(color=colorant"red")
    setobject!(vis["geom"], robot_obj, mat)
    settransform!(vis["geom"], compose(Translation(0,0,0.07),LinearMap( RotY(pi/2)*RotZ(-pi/2) )))
end

In [16]:
#Start the visualizer
vis = Visualizer()
TrajOptPlots.set_mesh!(vis, model)
render(vis)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8704
└ @ MeshCat /Users/kevintracy/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("robot" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x72, 0x6f, 0x62, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x72, 0x6f, 0x62, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))))), Set{Any}(), ip"127.0.0.1", 8704))

In [17]:
#Visualize the trajectory (you'll probably want to zoom out and pan a little bit)
X1 = [SVector{13}(x) for x in eachcol(xhist_rk4)];
visualize!(vis, model, Tf, X1)

In [40]:
#Let's try something more interesting. We're going to loop the glider.

#Try some different initial conditions to find some that successfully perform a loop.
#You'll probably want to increase the horizontal velocity and aileron command

v0x_loop = v0[1] #horizontal velocity (change this)
ele_loop = u0[2] #aileron input (change this)

### Solution (many possibilities)
v0x_loop = 25
ele_loop = .25

x0_loop = [-5; 0; 2; q0; v0x_loop; v0[2:3]; ω0] #start faster
u0_loop = [u0[1]; ele_loop; u0[3]] #pitch up more

xhist_mid = zeros(13,N)
xhist_mid[:,1] .= x0_loop

xhist_rk4 = zeros(13,N)
xhist_rk4[:,1] .= x0_loop

for k = 1:(N-1)
    xhist_mid[:,k+1] .= normalized_mid_step(xhist_mid[:,k],u0_loop)
    xhist_rk4[:,k+1] .= normalized_rk4_step(xhist_rk4[:,k],u0_loop)
end

X1 = [SVector{13}(x) for x in eachcol(xhist_rk4)];
visualize!(vis, model, Tf, X1)

In [41]:
#Let's also compute a reference solution using DifferentialEquations.jl
using OrdinaryDiffEq

#wrapper function for DifferentialEquations.jl interface 
function f(x,p,t)    
    ẋ = glider_dynamics_q(x,u0_loop)
end

tspan = (0.0,Tf)
prob = ODEProblem(f,x0_loop,tspan)
sol = solve(prob,Tsit5(),reltol=1e-12,abstol=1e-12);

xref = zeros(13,N)
for k = 1:N
    xref[:,k] = sol(tsamp[k])
end

In [42]:
#Now we're going to implement the glider dynamics using an SE(3) parameterization for the configuration

#The hat map for se(3) that maps [v; ω] into a 4x4 matrix. This will be useful for implementing the SE(3) kinematics
function hat_se3(x)
    [hat(x[4:6]) x[1:3]; zeros(1,4)]
end

function glider_dynamics_se3(T,w,u)
    #Unpack state vector
    r = T[1:3, 4]
    Q = T[1:3, 1:3]
    v = w[1:3] #B frame
    ω = w[4:6] #B frame
    
    ### Solution
    
    #Forces
    F_aero, τ_aero = aero_forces(v,ω,u)
    F = F_aero + Q'*[0; 0; -m*g]
    τ = τ_aero
    
    #Kinematics
    Ṫ = T*hat_se3(w)
    
    #Dynamics
    v̇ = F/m - hat(ω)*v
    ω̇ = J\(τ - hat(ω)*J*ω)
    ẇ = [v̇; ω̇]
    
    return Ṫ, ẇ
end

glider_dynamics_se3 (generic function with 1 method)

In [18]:
@testset "se3 dynamics" begin 
    
    x = [1;2;3;4;5;6;7]
    H1 = hat_se3(x)
    
    test_dict["H1"] = copy(H1)
    
    @test size(H1) == size(test_dict["H1"])
    @test isapprox(vec(H1),vec(test_dict["H1"]),atol = 1e-10)
end

LoadError: LoadError: UndefVarError: @testset not defined
in expression starting at In[18]:1

In [43]:
#Let's implement a lie-group midpoint integrator similar to the one in the first homework, but this time for SE(3) instead of SO(3)

function lie_midpoint_step(Tk,wk,uk)
    
    ### Solution
    Ṫ1, ẇ1 = glider_dynamics_se3(Tk,wk,uk)
    
    Tm = Tk*exp(0.5*h*hat_se3(wk)) # Tk + 0.5*h*Ṫ1 also works
    wm = wk + 0.5*h*ẇ1
    
    Ṫm, ẇm = glider_dynamics_se3(Tm,wm,uk)
    
    Tn = Tk*exp(h*hat_se3(wm))
    wn = wk + h*ẇm
    
    return Tn, wn
end

lie_midpoint_step (generic function with 1 method)

In [44]:
#Convert the initial conditions we had before into an SE(3) state

### Solution
Q0 = H'*L(q0)*R(q0)'*H
T0_loop = [Q0 x0_loop[1:3]; 0 0 0 1]
w0_loop = x0_loop[8:13]

6-element Vector{Float64}:
 25.0
  0.0
  0.5678625279339364
  0.0
  0.0
  0.0

In [45]:
#Run the same simulation with the new integrator

Thist_lie = zeros(4,4,N)
whist_lie = zeros(6,N)
Thist_lie[:,:,1] .= T0_loop
whist_lie[:,1] .= w0_loop
for k = 1:(N-1)
    Tn, wn = lie_midpoint_step(Thist_lie[:,:,k], whist_lie[:,k],u0_loop)
    Thist_lie[:,:,k+1] .= Tn
    whist_lie[:,k+1] .= wn
end

In [46]:
#Compare the solutions
err_pos_mid = zeros(N)
err_rot_mid = zeros(N)
err_pos_rk4 = zeros(N)
err_rot_rk4 = zeros(N)
err_pos_lie = zeros(N)
err_rot_lie = zeros(N)
for k = 1:N
    err_pos_mid[k] = norm(xhist_mid[1:3,k]-xref[1:3,k])
    err_pos_rk4[k] = norm(xhist_rk4[1:3,k]-xref[1:3,k])
    err_pos_lie[k] = norm(Thist_lie[1:3,4,k]-xref[1:3,k])
    
    Qk_mid = H'*L(xhist_mid[4:7,k])*R(xhist_mid[4:7,k])'*H
    Qk_rk4 = H'*L(xhist_rk4[4:7,k])*R(xhist_rk4[4:7,k])'*H
    Qk_lie = Thist_lie[1:3,1:3,k]
    Qk_ref = H'*L(xref[4:7,k])*R(xref[4:7,k])'*H
    
    err_rot_mid[k] = norm(log(Qk_ref'*Qk_mid))
    err_rot_rk4[k] = norm(log(Qk_ref'*Qk_rk4))
    err_rot_lie[k] = norm(log(Qk_ref'*Qk_lie))
end

In [47]:
plot(err_pos_mid)
plot!(err_pos_rk4)
plot!(err_pos_lie)

In [49]:
plot(err_rot_mid)
plot!(err_rot_rk4)
plot!(err_rot_lie)